In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from glob import glob
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from scipy import stats

In [2]:
mu_b = 5.7883818012 * 10**(-5)
mu_b

5.7883818012e-05

In [6]:
list_sweepVT = sorted(glob('*Voltage_Sweep.tsv.gz'))
list_sweepVT

['12-ML_WLS-80MHz-0,6uW-610-775nm_-15-+15V_1s.Voltage_Sweep.tsv.gz',
 '13-BG_WLS-80MHz-0,6uW-610-775nm_-15-+15V_1s.Voltage_Sweep.tsv.gz']

In [7]:
list_calibration = glob('*.wlen_to_px.tsv.gz')
list_calibration

['12-ML_WLS-80MHz-0,6uW-610-775nm_-15-+15V_1s.wlen_to_px.tsv.gz',
 '13-BG_WLS-80MHz-0,6uW-610-775nm_-15-+15V_1s.wlen_to_px.tsv.gz']

In [8]:
calibration = 1230.8 / (pd.read_csv(list_calibration[0], header=None).as_matrix().transpose())[0]

In [18]:
spec = pd.read_csv(list_sweepVT[0], sep='\t', header=None).as_matrix()
background = pd.read_csv(list_sweepVT[1], sep='\t', header=None).as_matrix()
DR = (spec - background)/background

In [19]:
output_notebook()

Loading BokehJS ...

In [23]:
y = DR[5]
pp = figure()
pp.line(calibration, y)
show(pp)

array([1.85623947, 1.85587281, 1.85550909, ..., 1.47222415, 1.47199701,
       1.47177171])

In [69]:
def fano(E, E0, G, q, A):
    e = (E - E0)/(G/2)
    return(A*((q + e)**2)/((1 + e**2)))
#def fano(E, E0, G, q):
#    e = (E - E0)/(G/2)
#    return(((q + e)**2)/((1 + e)**2))

In [105]:
def lorentz(x, x_n, G):
    X = (x_n - x)/(G/2)
    return 1/(1+X**2)
def lorentz_scale(x, x_n, G, A):
    X = (x_n - x)/(G/2)
    return A*(1/(1+X**2))
def lorentz_double(x, xn1, G1, A1, xn2, G2, A2):
    return(lorentz(x, xn1, G1, A1) + lorentz(x, xn2, G2, A2))
def lorentz_double_dip(x, xn1, xn2, G1, G2, A1, A2):
    return(lorentz_scale(x, xn1, G1, A1) + lorentz_scale(x, xn2, G2, A2))

In [16]:
a_min, a_max = 1.69, 1.72

In [113]:

# a: 1.695, 1.72 p0: 1.70, 0.05
# t_ind: 1.64, 1.66, 1.65, 0.05
# 3rd_n: 1.63, 1.642, 1.638, 0.05
# t: 1.666, 1.674, 
def fit_doubledip(data,
          cal=calibration,
          a_min=1.673,
          a_max=1.683,
          base_function=lorentz_double_dip,
          p0=[1.676, 1.682, 0.03, 0.03, 0.1, 0.1]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    #data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(base_function, cal_slice, data_slice, p0=p0)
    return popt

def fit_fano(data,
          cal=calibration,
          a_min=1.65,
          a_max=1.75,
          base_function=fano,
          p0=[1.7, 0.1, -1, 1]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    #data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(base_function, cal_slice, data_slice, p0=p0)
    return popt

def fit_peak(data,
          cal=calibration,
          a_min=1.673,
          a_max=1.682,
          base_function=lorentz,
          p0=[1.75, 0.02]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(lorentz, cal_slice, data_slice, p0=p0)
    return popt[0]

def fit_a(data,
          cal=calibration,
          a_min=1.695,
          a_max=1.72,
          base_function=lorentz,
          p0=[1.7, 0.05]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(lorentz, cal_slice, data_slice, p0=p0)
    return popt[0]
def fit_t(data,
          cal=calibration,
          a_min=1.66,
          a_max=1.69,
          base_function=lorentz,
          p0=[1.67, 0.05]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(lorentz, cal_slice, data_slice, p0=p0)
    return popt[0]
def fit_td(data,
          cal=calibration,
          a_min=1.638,
          a_max=1.654,
          base_function=lorentz,
          p0=[1.645, 0.05]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(lorentz, cal_slice, data_slice, p0=p0)
    return popt[0]
def fit_Tr(data,
          cal=calibration,
          a_min=1.62,#1.66,
          a_max=1.69,#1.69,
          base_function=lorentz_double,
          p0=[1.63, 0.05, 10, 1.67, 0.05, 10]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    #data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(base_function, cal_slice, data_slice, p0=p0)
    return popt


In [73]:
y = fano(calibration, 1.7, 0.1, 1, 1)
pp = figure()
pp.line(calibration, y)
show(pp)

In [118]:
y = DR[0] - min(DR[0])
a, b, c, d = fit_fano(y)
print(a)
print(b)
print(c)
print(d)
y2 = fano(calibration, a, b, c, d)
pp = figure()
pp.line(calibration, y)
pp.line(calibration, y2, color='red')
show(pp)

1.7099178468101783
0.011717613612611638
-1.9159661794536864
0.2466789179776302


In [117]:
y = DR[0]*(-1) + 0.26
a, b, c, d, e, f= fit_doubledip(y)
print(a)
print(b)
print(c)
print(d)
y2 = lorentz_double_dip(calibration, a, b, c, d, e, f)
pp = figure()
pp.line(calibration, y)
pp.line(calibration, y2, color='red')
show(pp)

1.6759006708084017
1.6817055631461157
-0.005918084132260119
0.003683702045104716
